In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/frames/Frames/frame_0004.jpg
/kaggle/input/frames/Frames/frame_0146.jpg
/kaggle/input/frames/Frames/frame_0099.jpg
/kaggle/input/frames/Frames/frame_0025.jpg
/kaggle/input/frames/Frames/frame_0023.jpg
/kaggle/input/frames/Frames/frame_0020.jpg
/kaggle/input/frames/Frames/frame_0183.jpg
/kaggle/input/frames/Frames/frame_0204.jpg
/kaggle/input/frames/Frames/frame_0138.jpg
/kaggle/input/frames/Frames/frame_0164.jpg
/kaggle/input/frames/Frames/frame_0015.jpg
/kaggle/input/frames/Frames/frame_0019.jpg
/kaggle/input/frames/Frames/frame_0067.jpg
/kaggle/input/frames/Frames/frame_0149.jpg
/kaggle/input/frames/Frames/frame_0080.jpg
/kaggle/input/frames/Frames/frame_0049.jpg
/kaggle/input/frames/Frames/frame_0189.jpg
/kaggle/input/frames/Frames/frame_0091.jpg
/kaggle/input/frames/Frames/frame_0089.jpg
/kaggle/input/frames/Frames/frame_0054.jpg
/kaggle/input/frames/Frames/frame_0142.jpg
/kaggle/input/frames/Frames/frame_0139.jpg
/kaggle/input/frames/Frames/frame_0119.jpg
/kaggle/inp

In [13]:
height, width = 1024,800

In [14]:
import os
import cv2
import numpy as np

# Define the directory containing the hazed images
hazy_folder = '/kaggle/input/revide-indoor/Train/hazy/C006_2/'

# Function to load and preprocess an image
def load_and_preprocess_image(image_path, target_height, target_width):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Ensure RGB format
    image = cv2.resize(image, (target_width, target_height))  # Resize to your desired dimensions
    image = image / 255.0  # Normalize pixel values to the range [0, 1]
    return image

# Load all hazed images from the subfolders
X_train = []

for root, subdirs, files in os.walk(hazy_folder):
    for filename in files:
        if filename.endswith(".JPG"):
            image_path = os.path.join(root, filename)
            img = load_and_preprocess_image(image_path, height, width)
            X_train.append(img)

# Convert the list of images to a NumPy array
X_train = np.array(X_train)



In [15]:
import os
import cv2
import numpy as np

# Define the directory containing the hazed images
unhazed_image_folder = '/kaggle/input/revide-indoor/Train/gt/C006_2/'

# Function to load and preprocess an image
def load_and_preprocess_image(image_path, target_height, target_width):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Ensure RGB format
    image = cv2.resize(image, (target_width, target_height))  # Resize to your desired dimensions
    image = image / 255.0  # Normalize pixel values to the range [0, 1]
    return image

# Load all hazed images from the folder
Y_train = []

for filename in os.listdir(unhazed_image_folder):
    if filename.endswith(".JPG"):
        image_path = os.path.join(unhazed_image_folder, filename)
        img = load_and_preprocess_image(image_path, height, width)
        Y_train.append(img)

# Convert the list of images to a NumPy array
Y_train = np.array(Y_train)

In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Input layer for the model
input_layer = keras.Input(shape=(height, width, 3))

# Encoder
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

# Residual Blocks
for _ in range(4):
    residual = x
    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([residual, x])

# Decoder
x = layers.UpSampling2D(size=(2, 2))(x)
x = layers.Conv2D(64, (3, 3), padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, (3, 3), padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.BatchNormalization()(x)
output_layer = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Create the model
model = keras.Model(inputs=input_layer, outputs=output_layer)

# Learning rate scheduling
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9
)

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_schedule), loss='mean_squared_error')

model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 1024, 800,   0           []                               
                                3)]                                                               
                                                                                                  
 conv2d_13 (Conv2D)             (None, 1024, 800, 6  1792        ['input_2[0][0]']                
                                4)                                                                
                                                                                                  
 batch_normalization_12 (BatchN  (None, 1024, 800, 6  256        ['conv2d_13[0][0]']              
 ormalization)                  4)                                                          

In [17]:
model.fit(X_train, Y_train, epochs=20, batch_size=1)


# Save the trained model
model.save('dehazing_model.h5')

Epoch 1/20
45/45 [==============================] - 15s 159ms/step - loss: 0.0404
Epoch 2/20
45/45 [==============================] - 7s 158ms/step - loss: 0.0109
Epoch 3/20
45/45 [==============================] - 7s 159ms/step - loss: 0.0066
Epoch 4/20
45/45 [==============================] - 7s 159ms/step - loss: 0.0058
Epoch 5/20
45/45 [==============================] - 7s 159ms/step - loss: 0.0062
Epoch 6/20
45/45 [==============================] - 7s 159ms/step - loss: 0.0049
Epoch 7/20
45/45 [==============================] - 7s 159ms/step - loss: 0.0050
Epoch 8/20
45/45 [==============================] - 7s 158ms/step - loss: 0.0053
Epoch 9/20
45/45 [==============================] - 7s 159ms/step - loss: 0.0048
Epoch 10/20
45/45 [==============================] - 7s 159ms/step - loss: 0.0053
Epoch 11/20
45/45 [==============================] - 7s 159ms/step - loss: 0.0049
Epoch 12/20
45/45 [==============================] - 7s 158ms/step - loss: 0.0041
Epoch 13/20
45/45 [=====

In [11]:
import cv2
import numpy as np
import os
from os.path import isfile, join
from tensorflow import keras

# Define the preprocessing function for dehazing
def preprocess_image(image, model_input_shape):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Ensure RGB format
    image = cv2.resize(image, (model_input_shape[2], model_input_shape[1]))  # Resize to match the model's input shape
    image = image / 255.0  # Normalize pixel values to the range [0, 1]
    return image

# Define the post-processing function for dehazing
def postprocess_image(image):
    image = (image * 255).astype(np.uint8)
    return image

# Load the trained model for dehazing
model = keras.models.load_model('dehazing_model.h5')

# Define the path for reading image frames and creating the video
pathIn = '/kaggle/input/frames/Frames'
pathOut = 'videoframes.avi'
fps = 30

frame_array = []

# Sort the file names properly
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
files.sort(key=lambda x: x[5:-4])

# Read each file, preprocess, dehaze, and store the frames
for i in range(len(files)):
    filename = pathIn + '/' + files[i]  # Add a '/' between the path and file name
    
    # Attempt to read and preprocess each frame
    frame = cv2.imread(filename)
    
    if frame is not None:
        frame = preprocess_image(frame, model.input_shape)
        
        # Predict the dehazed image
        dehazed_frame = model.predict(np.array([frame]))[0]
        
        # Post-process the dehazed image
        dehazed_frame = postprocess_image(dehazed_frame)
        
        # Append the dehazed frame to the array
        frame_array.append(dehazed_frame)
    else:
        print(f"Failed to read file: {filename}")

# Create the video writer
size = (frame_array[0].shape[1], frame_array[0].shape[0])
out = cv2.VideoWriter(pathOut, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

# Write each dehazed frame to the video
for i in range(len(frame_array)):
    out.write(frame_array[i])

# Release the video writer
out.release()


1/1 [==============================] - 0s 23ms/step


In [26]:
import cv2
import numpy as np
import os
from os.path import isfile, join
from tensorflow import keras

# Define the preprocessing function for dehazing
def preprocess_image(image, model_input_shape):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Ensure RGB format
    image = cv2.resize(image, (model_input_shape[2], model_input_shape[1]))  # Resize to match the model's input shape
    image = image / 255.0  # Normalize pixel values to the range [0, 1]
    return image

# Define the post-processing function for dehazing
def postprocess_image(image):
    image = (image * 255).astype(np.uint8)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    print(image.shape)
    return image

# Load the trained model for dehazing
model = keras.models.load_model('dehazing_model.h5')

# Define the path for reading image frames and creating the video
pathIn = '/kaggle/input/frames/Frames'
pathOut = 'videoframes1.avi'
fps = 30

frame_array = []

# Sort the file names properly
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
files.sort(key=lambda x: x[5:-4])

# Create the dehaze_image function
def dehaze_image(image):
    # Predict the dehazed image using the model
    dehazed_frame = model.predict(np.array([image]))[0]
    
    # Post-process the dehazed image
    dehazed_frame = postprocess_image(dehazed_frame)
    
    return dehazed_frame

# Read each file, preprocess, dehaze, and store the frames
for i in range(len(files)):
    filename = pathIn + '/' + files[i]  # Add a '/' between the path and file name
    
    # Attempt to read and preprocess each frame
    frame = cv2.imread(filename)
    
    if frame is not None:
        frame = preprocess_image(frame, model.input_shape)
        
        # Predict and process the dehazed image
        dehazed_frame = dehaze_image(frame)
        display(dehazed_frame)
        # Append the dehazed frame to the array
        frame_array.append(dehazed_frame)
    else:
        print(f"Failed to read file: {filename}")


# Create the video writer
size = (frame_array[0].shape[1], frame_array[0].shape[0])
out = cv2.VideoWriter(pathOut, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

# Write each dehazed frame to the video
for i in range(len(frame_array)):
    out.write(frame_array[i])

# Release the video writer
out.release()

1/1 [==============================] - 0s 240ms/step
(1024, 800, 3)


array([[[ 84, 104, 116],
        [ 69,  84, 103],
        [ 64,  77,  96],
        ...,
        [105, 123, 145],
        [101, 117, 138],
        [111, 129, 142]],

       [[ 50,  58,  71],
        [ 20,  27,  42],
        [ 14,  20,  32],
        ...,
        [ 62,  80, 109],
        [ 58,  74, 102],
        [ 80, 104, 111]],

       [[ 34,  37,  44],
        [  9,  12,  16],
        [  5,   7,  11],
        ...,
        [ 44,  59,  82],
        [ 42,  56,  79],
        [ 66,  87,  93]],

       ...,

       [[ 71,  80, 108],
        [ 54,  64, 104],
        [ 52,  63, 107],
        ...,
        [  5,   8,  10],
        [ 12,  19,  27],
        [ 53,  63,  66]],

       [[ 75,  83, 109],
        [ 64,  68, 107],
        [ 61,  66, 108],
        ...,
        [  8,  11,  15],
        [ 19,  26,  37],
        [ 68,  78,  77]],

       [[105, 102, 126],
        [ 99,  97, 123],
        [ 96,  95, 120],
        ...,
        [ 31,  37,  37],
        [ 48,  63,  63],
        [ 91, 107,  98]]

1/1 [==============================] - 0s 23ms/step
(1024, 800, 3)


array([[[ 84, 104, 116],
        [ 69,  84, 103],
        [ 64,  77,  96],
        ...,
        [105, 123, 145],
        [101, 117, 138],
        [111, 129, 142]],

       [[ 50,  58,  71],
        [ 20,  27,  42],
        [ 14,  20,  32],
        ...,
        [ 62,  80, 109],
        [ 58,  74, 102],
        [ 80, 104, 111]],

       [[ 34,  37,  44],
        [  9,  12,  16],
        [  5,   7,  11],
        ...,
        [ 44,  59,  82],
        [ 42,  56,  79],
        [ 66,  87,  93]],

       ...,

       [[ 71,  80, 108],
        [ 54,  64, 104],
        [ 52,  63, 107],
        ...,
        [  5,   8,  10],
        [ 12,  19,  27],
        [ 53,  63,  66]],

       [[ 75,  83, 109],
        [ 64,  68, 107],
        [ 61,  66, 108],
        ...,
        [  8,  11,  15],
        [ 19,  26,  37],
        [ 68,  78,  77]],

       [[105, 102, 126],
        [ 99,  97, 123],
        [ 96,  95, 120],
        ...,
        [ 31,  37,  37],
        [ 48,  63,  63],
        [ 91, 107,  98]]

1/1 [==============================] - 0s 23ms/step
(1024, 800, 3)


...

array([[[ 78,  96, 105],
        [ 58,  71,  86],
        [ 53,  63,  77],
        ...,
        [ 74,  91, 115],
        [ 81,  97, 122],
        [103, 121, 138]],

       [[ 42,  47,  55],
        [ 13,  18,  25],
        [  8,  11,  17],
        ...,
        [ 26,  37,  60],
        [ 35,  48,  74],
        [ 67,  91, 100]],

       [[ 25,  27,  30],
        [  5,   6,   8],
        [  2,   3,   4],
        ...,
        [ 18,  26,  40],
        [ 26,  36,  55],
        [ 59,  77,  84]],

       ...,

       [[ 27,  28,  35],
        [  6,   6,  11],
        [  4,   4,   7],
        ...,
        [ 11,  17,  23],
        [ 21,  31,  46],
        [ 65,  80,  84]],

       [[ 34,  34,  42],
        [ 10,  10,  17],
        [  7,   7,  11],
        ...,
        [ 16,  20,  30],
        [ 29,  38,  56],
        [ 79,  93,  93]],

       [[ 69,  66,  76],
        [ 41,  39,  47],
        [ 32,  32,  36],
        ...,
        [ 45,  49,  54],
        [ 61,  76,  79],
        [ 98, 117, 110]]

1/1 [==============================] - 0s 23ms/step
(1024, 800, 3)


array([[[ 78,  96, 105],
        [ 58,  71,  86],
        [ 53,  63,  77],
        ...,
        [ 74,  91, 115],
        [ 81,  97, 122],
        [103, 121, 138]],

       [[ 42,  47,  55],
        [ 13,  18,  25],
        [  8,  11,  17],
        ...,
        [ 26,  37,  60],
        [ 35,  48,  74],
        [ 67,  91, 100]],

       [[ 25,  27,  30],
        [  5,   6,   8],
        [  2,   3,   4],
        ...,
        [ 18,  26,  40],
        [ 26,  36,  55],
        [ 59,  77,  84]],

       ...,

       [[ 27,  28,  35],
        [  6,   6,  11],
        [  4,   4,   7],
        ...,
        [ 11,  17,  23],
        [ 21,  31,  46],
        [ 65,  80,  84]],

       [[ 34,  34,  42],
        [ 10,  10,  17],
        [  7,   7,  11],
        ...,
        [ 16,  20,  30],
        [ 29,  38,  56],
        [ 79,  93,  93]],

       [[ 69,  66,  76],
        [ 41,  39,  47],
        [ 32,  32,  36],
        ...,
        [ 45,  49,  54],
        [ 61,  76,  79],
        [ 98, 117, 110]]

1/1 [==============================] - 0s 23ms/step
(1024, 800, 3)


array([[[ 78,  96, 105],
        [ 58,  71,  86],
        [ 53,  63,  77],
        ...,
        [ 74,  91, 115],
        [ 81,  97, 122],
        [103, 121, 138]],

       [[ 42,  47,  55],
        [ 13,  18,  25],
        [  8,  11,  17],
        ...,
        [ 26,  37,  60],
        [ 35,  48,  74],
        [ 67,  91, 100]],

       [[ 25,  27,  30],
        [  5,   6,   8],
        [  2,   3,   4],
        ...,
        [ 18,  26,  40],
        [ 26,  36,  55],
        [ 59,  77,  84]],

       ...,

       [[ 27,  28,  35],
        [  6,   6,  11],
        [  4,   4,   7],
        ...,
        [ 11,  17,  23],
        [ 21,  31,  46],
        [ 65,  80,  84]],

       [[ 34,  34,  42],
        [ 10,  10,  17],
        [  7,   7,  11],
        ...,
        [ 16,  20,  30],
        [ 29,  38,  56],
        [ 79,  93,  93]],

       [[ 69,  66,  76],
        [ 41,  39,  47],
        [ 32,  32,  36],
        ...,
        [ 45,  49,  54],
        [ 61,  76,  79],
        [ 98, 117, 110]]

In [22]:
import cv2
import numpy as np
import os
from os.path import isfile, join
from tensorflow import keras

# Define the preprocessing function for dehazing
def preprocess_image(image, model_input_shape):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Ensure RGB format
    image = cv2.resize(image, (model_input_shape[2], model_input_shape[1]))  # Resize to match the model's input shape
    image = image / 255.0  # Normalize pixel values to the range [0, 1]
    return image

# Define the post-processing function for dehazing
def postprocess_image(image):
    image = (image * 255).astype(np.uint8)
    return image

# Load the trained model for dehazing
model = keras.models.load_model('dehazing_model.h5')

# Define the path for reading image frames and creating the video
pathIn = '/kaggle/input/frames/Frames'
pathOut = 'videoframes2.avi'
fps = 30

frame_array = []

# Sort the file names properly
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
files.sort(key=lambda x: x[5:-4])

# Create the dehaze_image function
def dehaze_image(image):
    # Predict the dehazed image using the model
    dehazed_frame = model.predict(np.array([image]))[0]
    
    # Post-process the dehazed image
    dehazed_frame = postprocess_image(dehazed_frame)
    
    return dehazed_frame

# Read each file, preprocess, dehaze, and store the frames
for i in range(len(files)):
    filename = pathIn + '/' + files[i]  # Add a '/' between the path and file name
    
    # Attempt to read and preprocess each frame
    frame = cv2.imread(filename)
    
    if frame is not None:
        frame = preprocess_image(frame, model.input_shape)
        
        # Predict and process the dehazed image
        dehazed_frame = dehaze_image(frame)
        
        # Append the dehazed frame to the array
        frame_array.append(dehazed_frame)
    else:
        print(f"Failed to read file: {filename}")

# Create the video writer
size = (frame_array[0].shape[1], frame_array[0].shape[0])
out = cv2.VideoWriter(pathOut, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

# Write each dehazed frame to the video
for i in range(len(frame_array)):
    out.write(frame_array[i])

# Release the video writer
out.release()

1/1 [==============================] - 0s 24ms/step


In [ ]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt
# #0.0210
# # Define a preprocessing function
# def preprocess_image(image):
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Ensure RGB format
#     image = cv2.resize(image, (width, height))  # Resize to your desired dimensions
#     image = image / 255.0  # Normalize pixel values to the range [0, 1]
#     return image

# # Define a post-processing function
# def postprocess_image(image):
#     image = (image * 255).astype(np.uint8)
#     return image

# # Load the trained model
# model = keras.models.load_model('dehazing_model.h5')

# # Load a hazed test image
# hazed_test_image = cv2.imread('/kaggle/input/revide-indoor/Test/hazy/L006/00029.JPG')

# # Preprocess and resize the test image
# hazed_test_image = preprocess_image(hazed_test_image)

# # Predict the dehazed image
# dehazed_test_image = model.predict(np.array([hazed_test_image]))[0]

# # Post-process the dehazed image
# dehazed_test_image = postprocess_image(dehazed_test_image)

# Display the dehazed image
plt.imshow(dehazed_test_image)
plt.axis('off')  # Turn off axis labels
plt.show()